In [ ]:
#추진방법
##이미지, 영상에서 사람얼굴 식별(Opencv, Haar-Cascade-Classifiers, 딥러닝 등)
##블러처리 하여 export
##향후 streamlit, bootstrap 등으로 웹으로 시각화하여 구축
##최종적으로 이미지, 영상 업로드하여 가명처리된 이미지, 영상 export 설계
##영상 처리시 속도가 너무 느림, 경량화하는 방법 필요

#참고문헌
## https://velog.io/@smile_b/Haar-Cascade-Classifiers%EB%A5%BC-%EC%9D%B4%EC%9A%A9%ED%95%9C-%EC%96%BC%EA%B5%B4-%EB%B0%8F-%EB%88%88-%EA%B0%90%EC%A7%80

In [1]:
#오픈 CV 설치
!pip install opencv-python

In [2]:
#현재 경로파악
!ls

'ls'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


### 효율적인 블러처리 방법 찾기

In [14]:
import torch

In [15]:
print(torch.cuda.is_available())

True


In [1]:
pip install opencv-python

  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [33]:
import cv2
print(cv2.getBuildInformation())


General configuration for OpenCV 4.10.0 =====================================
  Version control:               4.10.0

  Platform:
    Timestamp:                   2024-06-17T17:58:36Z
    Host:                        Windows 10.0.17763 AMD64
    CMake:                       3.24.2
    CMake generator:             Visual Studio 14 2015
    CMake build tool:            MSBuild.exe
    MSVC:                        1900
    Configuration:               Debug Release

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (16 files):         + SSSE3 SSE4_1
      SSE4_2 (1 files):          + SSSE3 SSE4_1 POPCNT SSE4_2
      FP16 (0 files):            + SSSE3 SSE4_1 POPCNT SSE4_2 FP16 AVX
      AVX (8 files):             + SSSE3 SSE4_1 POPCNT SSE4_2 AVX
      AVX2 (36 files):           + SSSE3 SSE4_

In [2]:
#1-3 가장 최근에 나온 YOLOv8-Face로 활용
!pip install ultralytics

   ---------------------------------------- 0.0/879.0 kB ? eta -:--:--
   ---------------------------------------- 879.0/879.0 kB 7.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ------------ --------------------------- 2.4/7.8 MB 11.2 MB/s eta 0:00:01
   ------------------------- -------------- 5.0/7.8 MB 11.6 MB/s eta 0:00:01
   ------------------------------------- -- 7.3/7.8 MB 11.6 MB/s eta 0:00:01
   ---------------------------------------- 7.8/7.8 MB 10.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   -------- ------------------------------- 2.4/11.6 MB 12.2 MB/s eta 0:00:01
   ---------------- ----------------------- 4.7/11.6 MB 11.9 MB/s eta 0:00:01
   ------------------------- -------------- 7.3/11.6 MB 11.9 MB/s eta 0:00:01
   ---------------------------------- ----- 10.0/11.6 MB 11.9 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 11.2 MB/s eta 0:00:00
   -------

In [ ]:
import cv2
import torch
from ultralytics import YOLO
import time

# 파일 경로 설정
input_video_path = 'content/sample_data/sample.mp4'
output_video_path = 'content/sample_data/blurred_output.mp4'

# YOLOv11s-Face 모델 로드 (GPU 사용 설정)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = YOLO("content/weights/yolov11s-face.pt").to(device)

# 동영상 파일 열기
cap = cv2.VideoCapture(input_video_path)

# 동영상의 속성 가져오기
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))


# 비디오 저장 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# 시간 측정 시작
start_time = time.time()
frame_count = 0
frame_skip = 5  # 프레임 샘플링 간격 (5 프레임마다 얼굴 재탐지)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 프레임 샘플링: frame_skip 간격으로 얼굴 탐지 수행
    if frame_count % frame_skip == 0:
        # YOLOv8-Face로 얼굴 탐지 (GPU에서 실행)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # RGB 변환 (필요 시)
        results = model(frame_rgb)

        # 얼굴 영역을 저장
        faces = []
        for result in results:
            boxes = result.boxes.xyxy  # 얼굴 바운딩 박스 좌표 (x1, y1, x2, y2)
            for box in boxes:
                x1, y1, x2, y2 = map(int, box)  # 정수형으로 변환
                faces.append((x1, y1, x2, y2))  # 얼굴 위치 저장

    # 이전에 저장된 얼굴 위치에 블러 처리
    for (x1, y1, x2, y2) in faces:
        face = frame[y1:y2, x1:x2]
        blurred_face = cv2.GaussianBlur(face, (25, 25), 50)  # 강한 블러 적용
        frame[y1:y2, x1:x2] = blurred_face

    # 결과 프레임을 출력 파일에 저장
    out.write(frame)
    frame_count += 1

    # 남은 시간 예측
    if frame_count % 10 == 0:  # 매 10 프레임마다 남은 시간 업데이트
        elapsed_time = time.time() - start_time
        estimated_total_time = (elapsed_time / frame_count) * total_frames
        remaining_time = estimated_total_time - elapsed_time
        print(f"진행률: {frame_count / total_frames * 100:.2f}% | 남은 시간: {remaining_time:.2f}초")

# 자원 해제
cap.release()
out.release()
cv2.destroyAllWindows()

# 총 처리 시간
end_time = time.time()
total_elapsed_time = end_time - start_time

print("가명처리된 동영상이 저장되었습니다:", output_video_path)
print(f"총 처리 시간: {total_elapsed_time:.2f}초")


0: 384x640 (no detections), 17.0ms
Speed: 5.0ms preprocess, 17.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
진행률: 0.83% | 남은 시간: 94.57초

0: 384x640 (no detections), 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
진행률: 1.67% | 남은 시간: 65.03초

0: 384x640 (no detections), 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
진행률: 2.50% | 남은 시간: 54.74초

0: 384x640 (no detections), 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3

### frame skip 동적으로 계산

동영상 불러오기
동영상 저장 시간 줄이기

In [ ]:
import cv2
import torch
from ultralytics import YOLO
import time

# 파일 경로 설정
input_video_path = 'content/sample_data/sample.mp4'
output_video_path = 'content/sample_data/blurred_output.mp4'

# YOLOv11s-Face 모델 로드 (GPU 사용 설정)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_load_start = time.time()
model = YOLO("content/weights/yolov11s-face.pt").to(device)
model_load_end = time.time()
print(f"모델 로드 시간: {model_load_end - model_load_start:.8f}초")

# 동영상 파일 열기
video_open_start = time.time()
cap = cv2.VideoCapture(input_video_path)
video_open_end = time.time()
print(f"동영상 파일 열기 시간: {video_open_end - video_open_start:.8f}초")

# 동영상의 속성 가져오기
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# 비디오 저장 설정
video_writer_start = time.time()
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
video_writer_end = time.time()
print(f"비디오 저장 설정 시간: {video_writer_end - video_writer_start:.8f}초")

# 시간 측정 시작
start_time = time.time()
frame_count = 0

# 초당 프레임 수의 20%를 기반으로 frame_skip 값 설정
frame_skip = max(int(fps * 0.2), 1)  # 최소 frame_skip 값은 1로 설정
print(f"동적 설정된 frame_skip 값: {frame_skip}")

# 얼굴 위치를 저장할 리스트 (재탐지하기 전까지 사용)
faces = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 프레임 샘플링: frame_skip 간격으로 얼굴 탐지 수행
    if frame_count % frame_skip == 0:
        detection_start = time.time()
        # YOLOv11s-Face로 얼굴 탐지
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # RGB 변환 (필요 시)
        results = model(frame_rgb)
        detection_end = time.time()
        print(f"프레임 {frame_count} 얼굴 탐지 시간: {detection_end - detection_start:.8f}초")

        # 얼굴 영역을 저장
        faces = []  # 새로 탐지된 얼굴 위치로 업데이트
        for result in results:
            boxes = result.boxes.xyxy  # 얼굴 바운딩 박스 좌표 (x1, y1, x2, y2)
            for box in boxes:
                x1, y1, x2, y2 = map(int, box)  # 정수형으로 변환
                faces.append((x1, y1, x2, y2))  # 얼굴 위치 저장

    # 저장된 얼굴 위치에 블러 처리
    blur_start = time.time()
    for (x1, y1, x2, y2) in faces:
        face = frame[y1:y2, x1:x2]
        blurred_face = cv2.GaussianBlur(face, (25, 25), 50)  # 강한 블러 적용
        frame[y1:y2, x1:x2] = blurred_face
    blur_end = time.time()
    print(f"프레임 {frame_count} 블러 처리 시간: {blur_end - blur_start:.8f}초")

    # 결과 프레임을 출력 파일에 저장
    write_start = time.time()
    out.write(frame)
    write_end = time.time()
    print(f"프레임 {frame_count} 저장 시간: {write_end - write_start:.8f}초")

    frame_count += 1

    # 남은 시간 예측 (옵션)
    if frame_count % 10 == 0:  # 매 10 프레임마다 남은 시간 업데이트
        elapsed_time = time.time() - start_time
        estimated_total_time = (elapsed_time / frame_count) * total_frames
        remaining_time = estimated_total_time - elapsed_time
        print(f"진행률: {frame_count / total_frames * 100:.8f}% | 남은 시간: {remaining_time:.8f}초")

# 자원 해제
release_start = time.time()
cap.release()
out.release()
cv2.destroyAllWindows()
release_end = time.time()
print(f"자원 해제 시간: {release_end - release_start:.8f}초")

# 총 처리 시간
end_time = time.time()
total_elapsed_time = end_time - start_time

print("가명처리된 동영상이 저장되었습니다:", output_video_path)
print(f"총 처리 시간: {total_elapsed_time:.8f}초")


모델 로드 시간: 0.18918562초
동영상 파일 열기 시간: 0.05500102초
비디오 저장 설정 시간: 0.00299931초
동적 설정된 frame_skip 값: 12

0: 384x640 (no detections), 16.0ms
Speed: 5.0ms preprocess, 16.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
프레임 0 얼굴 탐지 시간: 0.39200449초
프레임 0 블러 처리 시간: 0.00000000초
프레임 0 저장 시간: 0.02499962초
프레임 1 블러 처리 시간: 0.00000000초
프레임 1 저장 시간: 0.02301145초
프레임 2 블러 처리 시간: 0.00000000초
프레임 2 저장 시간: 0.01800108초
프레임 3 블러 처리 시간: 0.00000000초
프레임 3 저장 시간: 0.01400065초
프레임 4 블러 처리 시간: 0.00000000초
프레임 4 저장 시간: 0.01399970초
프레임 5 블러 처리 시간: 0.00000000초
프레임 5 저장 시간: 0.01300001초
프레임 6 블러 처리 시간: 0.00000000초
프레임 6 저장 시간: 0.01500010초
프레임 7 블러 처리 시간: 0.00000000초
프레임 7 저장 시간: 0.01400280초
프레임 8 블러 처리 시간: 0.00000000초
프레임 8 저장 시간: 0.01500010초
프레임 9 블러 처리 시간: 0.00000000초
프레임 9 저장 시간: 0.01599979초
진행률: 0.83472454% | 남은 시간: 82.56633196초
프레임 10 블러 처리 시간: 0.00000000초
프레임 10 저장 시간: 0.01500058초
프레임 11 블러 처리 시간: 0.00000000초
프레임 11 저장 시간: 0.01400042초

0: 384x640 (no detections), 12.0ms
Speed: 3.0ms preprocess, 1

: 

In [2]:
pip install ffmpeg-python


  Using cached ffmpeg_python-0.2.0-py3-none-any.whl.metadata (1.7 kB)
Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)


In [ ]:
import cv2
import torch
from ultralytics import YOLO
import ffmpeg
import numpy as np
import time

# 파일 경로 설정
input_video_path = 'content/sample_data/sample.mp4'
output_video_path = 'content/sample_data/blurred_output.mp4'

# YOLOv11s-Face 모델 로드 (GPU 사용 설정)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_load_start = time.time()
model = YOLO("content/weights/yolov11s-face.pt").to(device)
model_load_end = time.time()
print(f"모델 로드 시간: {model_load_end - model_load_start:.4f}초")


# 동영상 파일 열기 (FFmpeg 사용)
video_open_start = time.time()
video_info = ffmpeg.probe(input_video_path)
video_stream = next((stream for stream in video_info['streams'] if stream['codec_type'] == 'video'), None)
if video_stream is None:
    raise ValueError("비디오 스트림을 찾을 수 없습니다.")

frame_width = int(video_stream['width'])
frame_height = int(video_stream['height'])
fps = eval(video_stream['r_frame_rate'])
total_frames = int(video_stream.get('nb_frames', 0))  # nb_frames가 없을 수 있으므로 기본값 설정
video_open_end = time.time()
print(f"동영상 파일 열기 시간: {video_open_end - video_open_start:.4f}초")
print(f"해상도: {frame_width}x{frame_height}, FPS: {fps}, 총 프레임 수: {total_frames}")

# 입력 스트림 설정
input_stream_start = time.time()
process_in = (
    ffmpeg
    .input(input_video_path)
    .output('pipe:', format='rawvideo', pix_fmt='bgr24')
    .run_async(pipe_stdout=True)
)
input_stream_end = time.time()
print(f"입력 스트림 설정 시간: {input_stream_end - input_stream_start:.4f}초")

# 출력 스트림 설정
output_stream_start = time.time()
process_out = (
    ffmpeg
    .input('pipe:', format='rawvideo', pix_fmt='bgr24', s='{}x{}'.format(frame_width, frame_height))
    .output(output_video_path, pix_fmt='yuv420p', vcodec='libx264', r=fps)
    .overwrite_output()
    .run_async(pipe_stdin=True)
)
output_stream_end = time.time()
print(f"출력 스트림 설정 시간: {output_stream_end - output_stream_start:.4f}초")

# 시간 측정 시작
start_time = time.time()
frame_count = 0

# 초당 프레임 수의 20%를 기반으로 frame_skip 값 설정
frame_skip = max(int(fps * 0.2), 1)  # 최소 frame_skip 값은 1로 설정
print(f"동적 설정된 frame_skip 값: {frame_skip}")

# 얼굴 위치를 저장할 리스트 (재탐지하기 전까지 사용)
faces = []

while True:
    frame_read_start = time.time()
    in_bytes = process_in.stdout.read(frame_width * frame_height * 3)
    if not in_bytes:
        break
    frame_read_end = time.time()
    print(f"프레임 {frame_count} 읽기 시간: {frame_read_end - frame_read_start:.4f}초")

    frame_process_start = time.time()
    frame = (
        np
        .frombuffer(in_bytes, np.uint8)
        .reshape([frame_height, frame_width, 3])
    )
    frame = frame.copy()  # 프레임 데이터를 수정 가능하도록 설정하기 위해 복사
    frame_process_end = time.time()
    print(f"프레임 {frame_count} 데이터 변환 시간: {frame_process_end - frame_process_start:.4f}초")

    # 프레임 샘플링: frame_skip 간격으로 얼굴 탐지 수행
    if frame_count % frame_skip == 0:
        detection_start = time.time()
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = model(frame_rgb)
        detection_end = time.time()
        print(f"프레임 {frame_count} 얼굴 탐지 시간: {detection_end - detection_start:.4f}초")

        # 얼굴 영역을 저장
        faces = []
        for result in results:
            boxes = result.boxes.xyxy
            for box in boxes:
                x1, y1, x2, y2 = map(int, box)
                faces.append((x1, y1, x2, y2))

    # 저장된 얼굴 위치에 블러 처리
    blur_start = time.time()
    for (x1, y1, x2, y2) in faces:
        face = frame[y1:y2, x1:x2]
        if face.size != 0:
            blurred_face = cv2.GaussianBlur(face, (25, 25), 50)
            frame[y1:y2, x1:x2] = blurred_face
    blur_end = time.time()
    print(f"프레임 {frame_count} 블러 처리 시간: {blur_end - blur_start:.4f}초")

    # 결과 프레임을 출력 스트림에 전달
    frame_write_start = time.time()
    process_out.stdin.write(
        frame
        .astype(np.uint8)
        .tobytes()
    )
    frame_write_end = time.time()
    print(f"프레임 {frame_count} 쓰기 시간: {frame_write_end - frame_write_start:.4f}초")

    frame_count += 1

    # 남은 시간 예측 (옵션)
    if frame_count % 10 == 0:
        elapsed_time = time.time() - start_time
        if total_frames > 0:
            estimated_total_time = (elapsed_time / frame_count) * total_frames
            remaining_time = estimated_total_time - elapsed_time
            progress = (frame_count / total_frames) * 100
            print(f"진행률: {progress:.4f}% | 남은 시간: {remaining_time:.4f}초")
        else:
            print(f"처리된 프레임 수: {frame_count} | 경과 시간: {elapsed_time:.4f}초")

# 프로세스 종료
process_end_start = time.time()
process_in.stdout.close()
process_out.stdin.close()
process_in.wait()
process_out.wait()
process_end_end = time.time()
print(f"프로세스 종료 시간: {process_end_end - process_end_start:.4f}초")

# 총 처리 시간
end_time = time.time()
total_elapsed_time = end_time - start_time

print("가명처리된 동영상이 저장되었습니다:", output_video_path)
print(f"총 처리 시간: {total_elapsed_time:.4f}초")


c:\Users\admin\anaconda3\envs\torch_GPU\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


모델 로드 시간: 0.2770초
동영상 파일 열기 시간: 0.0740초
해상도: 2560x1440, FPS: 60.0, 총 프레임 수: 1198
입력 스트림 설정 시간: 0.0180초
출력 스트림 설정 시간: 0.0200초
동적 설정된 frame_skip 값: 18
프레임 0 읽기 시간: 0.2220초
프레임 0 데이터 변환 시간: 0.0050초

0: 384x640 (no detections), 17.0ms
Speed: 6.0ms preprocess, 17.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
프레임 0 얼굴 탐지 시간: 4.0388초
프레임 0 블러 처리 시간: 0.0000초
프레임 0 쓰기 시간: 0.0180초
프레임 1 읽기 시간: 0.0200초
프레임 1 데이터 변환 시간: 0.0050초
프레임 1 블러 처리 시간: 0.0000초
프레임 1 쓰기 시간: 0.0120초
프레임 2 읽기 시간: 0.0210초
프레임 2 데이터 변환 시간: 0.0071초
프레임 2 블러 처리 시간: 0.0000초
프레임 2 쓰기 시간: 0.0139초
프레임 3 읽기 시간: 0.0210초
프레임 3 데이터 변환 시간: 0.0070초
프레임 3 블러 처리 시간: 0.0000초
프레임 3 쓰기 시간: 0.0090초
프레임 4 읽기 시간: 0.0200초
프레임 4 데이터 변환 시간: 0.0060초
프레임 4 블러 처리 시간: 0.0000초
프레임 4 쓰기 시간: 0.0080초
프레임 5 읽기 시간: 0.0220초
프레임 5 데이터 변환 시간: 0.0060초
프레임 5 블러 처리 시간: 0.0000초
프레임 5 쓰기 시간: 0.0090초
프레임 6 읽기 시간: 0.0190초
프레임 6 데이터 변환 시간: 0.0070초
프레임 6 블러 처리 시간: 0.0000초
프레임 6 쓰기 시간: 0.0090초
프레임 7 읽기 시간: 0.0200초
프레임 7 데이터 변환 시간: 0.0060초
프레임 7 블러 처리